# Parser d'Avis Critiques

Ce module implémente un parser pour extraire les livres et métadonnées des avis critiques au format markdown.

In [ ]:
# | default_exp avis_critiques_parser

In [ ]:
# | export
import re
import logging
from dataclasses import dataclass
from typing import List, Optional, Dict, Any
from bs4 import BeautifulSoup

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class ParsingError(Exception):
    """Exception levée lors d'erreurs de parsing des avis critiques."""
    pass


@dataclass
class BookMention:
    """Représente la mention d'un livre dans un avis critique."""
    titre: str
    auteur: str
    editeur: Optional[str] = None
    note: Optional[float] = None
    section: str = "programme"  # "programme" ou "coups_de_coeur"
    commentaire: Optional[str] = None
    nb_critiques: Optional[int] = None
    coup_de_coeur: Optional[str] = None
    chef_doeuvre: Optional[str] = None

In [ ]:
# | export
class AvisCritiquesParser:
    """Parser pour extraire les livres et métadonnées des avis critiques au format markdown."""
    
    def __init__(self):
        self.logger = logging.getLogger(__name__)
    
    def extract_books_from_summary(self, summary_text: str) -> List[BookMention]:
        """
        Extrait les livres d'un résumé d'avis critique.
        
        Args:
            summary_text (str): Le texte markdown de l'avis critique
            
        Returns:
            List[BookMention]: Liste des livres extraits
            
        Raises:
            ParsingError: Si le format est invalide ou inattendu
        """
        if not summary_text or not isinstance(summary_text, str):
            raise ParsingError("Le texte de résumé est vide ou invalide")
        
        books = []
        
        try:
            # Extraire les livres du programme principal
            programme_books = self._extract_programme_books(summary_text)
            books.extend(programme_books)
            
            # Extraire les coups de cœur
            coups_de_coeur_books = self._extract_coups_de_coeur_books(summary_text)
            books.extend(coups_de_coeur_books)
            
            self.logger.info(f"Extraction terminée: {len(books)} livres trouvés")
            return books
            
        except Exception as e:
            self.logger.error(f"Erreur lors de l'extraction des livres: {e}")
            raise ParsingError(f"Erreur lors de l'extraction: {e}")
    
    def _extract_programme_books(self, text: str) -> List[BookMention]:
        """Extrait les livres de la section 'LIVRES DISCUTÉS AU PROGRAMME'."""
        section_pattern = r"## 1\. LIVRES DISCUTÉS AU PROGRAMME.*?\n(.*?)(?=## 2\.|$)"
        match = re.search(section_pattern, text, re.DOTALL | re.IGNORECASE)
        
        if not match:
            self.logger.warning("Section 'LIVRES DISCUTÉS AU PROGRAMME' non trouvée")
            return []
        
        table_content = match.group(1)
        return self._parse_programme_table(table_content)
    
    def _extract_coups_de_coeur_books(self, text: str) -> List[BookMention]:
        """Extrait les livres de la section 'COUPS DE CŒUR DES CRITIQUES'."""
        section_pattern = r"## 2\. COUPS DE CŒUR DES CRITIQUES.*?\n(.*?)$"
        match = re.search(section_pattern, text, re.DOTALL | re.IGNORECASE)
        
        if not match:
            self.logger.warning("Section 'COUPS DE CŒUR DES CRITIQUES' non trouvée")
            return []
        
        table_content = match.group(1)
        return self._parse_coups_de_coeur_table(table_content)
    
    def _parse_programme_table(self, table_content: str) -> List[BookMention]:
        """Parse le tableau de la section programme principal."""
        books = []
        lines = table_content.strip().split('\n')
        
        # Trouver le début du tableau (ligne avec les headers)
        header_found = False
        data_lines = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # Détecter la ligne d'en-tête
            if '|' in line and ('Auteur' in line or 'Titre' in line):
                header_found = True
                continue
            
            # Ignorer la ligne de séparation
            if header_found and line.startswith('|') and '-' in line:
                continue
            
            # Collecter les lignes de données
            if header_found and line.startswith('|'):
                data_lines.append(line)
        
        # Parser chaque ligne de données
        for line in data_lines:
            try:
                book = self._parse_programme_table_row(line)
                if book:
                    books.append(book)
            except Exception as e:
                self.logger.warning(f"Erreur parsing ligne programme: {line[:50]}... - {e}")
                continue
        
        return books
    
    def _parse_programme_table_row(self, row: str) -> Optional[BookMention]:
        """Parse une ligne du tableau programme principal."""
        # Diviser la ligne en colonnes
        columns = [col.strip() for col in row.split('|')[1:-1]]  # Ignorer les | de début/fin
        
        if len(columns) < 4:  # Au minimum: Auteur, Titre, Éditeur, Avis
            return None
        
        auteur = columns[0].strip()
        titre = columns[1].strip()
        editeur = columns[2].strip() if columns[2].strip() else None
        avis_details = columns[3].strip() if len(columns) > 3 else ""
        
        # Extraire la note moyenne (colonne 4)
        note = None
        if len(columns) > 4:
            note = self._extract_rating(columns[4])
        
        # Extraire le nombre de critiques (colonne 5)
        nb_critiques = None
        if len(columns) > 5:
            nb_critiques = self._extract_number(columns[5])
        
        # Extraire coup de cœur (colonne 6)
        coup_de_coeur = None
        if len(columns) > 6:
            coup_de_coeur = columns[6].strip() if columns[6].strip() else None
        
        # Extraire chef d'œuvre (colonne 7)
        chef_doeuvre = None
        if len(columns) > 7:
            chef_doeuvre = columns[7].strip() if columns[7].strip() else None
        
        if not auteur or not titre:
            return None
        
        return BookMention(
            auteur=auteur,
            titre=titre,
            editeur=editeur,
            note=note,
            section="programme",
            commentaire=avis_details,
            nb_critiques=nb_critiques,
            coup_de_coeur=coup_de_coeur,
            chef_doeuvre=chef_doeuvre
        )
    
    def _parse_coups_de_coeur_table(self, table_content: str) -> List[BookMention]:
        """Parse le tableau de la section coups de cœur."""
        books = []
        lines = table_content.strip().split('\n')
        
        # Trouver le début du tableau
        header_found = False
        data_lines = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # Détecter la ligne d'en-tête
            if '|' in line and ('Auteur' in line or 'Titre' in line):
                header_found = True
                continue
            
            # Ignorer la ligne de séparation
            if header_found and line.startswith('|') and '-' in line:
                continue
            
            # Collecter les lignes de données
            if header_found and line.startswith('|'):
                data_lines.append(line)
        
        # Parser chaque ligne de données
        for line in data_lines:
            try:
                book = self._parse_coups_de_coeur_table_row(line)
                if book:
                    books.append(book)
            except Exception as e:
                self.logger.warning(f"Erreur parsing ligne coups de cœur: {line[:50]}... - {e}")
                continue
        
        return books
    
    def _parse_coups_de_coeur_table_row(self, row: str) -> Optional[BookMention]:
        """Parse une ligne du tableau coups de cœur."""
        # Format: | Auteur | Titre | Éditeur | Critique | Note | Commentaire |
        columns = [col.strip() for col in row.split('|')[1:-1]]
        
        if len(columns) < 3:  # Au minimum: Auteur, Titre, Éditeur
            return None
        
        auteur = columns[0].strip()
        titre = columns[1].strip()
        editeur = columns[2].strip() if columns[2].strip() else None
        
        # Critique (colonne 3)
        critique = columns[3].strip() if len(columns) > 3 else None
        
        # Note (colonne 4)
        note = None
        if len(columns) > 4:
            note = self._extract_rating(columns[4])
        
        # Commentaire (colonne 5)
        commentaire = columns[5].strip() if len(columns) > 5 else None
        
        if not auteur or not titre:
            return None
        
        return BookMention(
            auteur=auteur,
            titre=titre,
            editeur=editeur,
            note=note,
            section="coups_de_coeur",
            commentaire=commentaire,
            coup_de_coeur=critique
        )
    
    def _extract_rating(self, rating_html: str) -> Optional[float]:
        """
        Extrait la note numérique du HTML coloré.
        
        Args:
            rating_html (str): HTML contenant la note avec style coloré
            
        Returns:
            Optional[float]: La note extraite ou None si non trouvée
        """
        if not rating_html:
            return None
        
        try:
            # Essayer d'extraire directement un nombre
            number_match = re.search(r'(\d+\.?\d*)', rating_html)
            if number_match:
                return float(number_match.group(1))
            
            # Essayer d'extraire depuis du HTML avec BeautifulSoup
            soup = BeautifulSoup(rating_html, 'html.parser')
            text = soup.get_text().strip()
            
            # Chercher un nombre dans le texte extrait
            number_match = re.search(r'(\d+\.?\d*)', text)
            if number_match:
                return float(number_match.group(1))
            
            return None
            
        except (ValueError, AttributeError) as e:
            self.logger.warning(f"Impossible d'extraire la note de '{rating_html}': {e}")
            return None
    
    def _extract_number(self, text: str) -> Optional[int]:
        """Extrait un nombre entier d'un texte."""
        if not text:
            return None
        
        try:
            # Chercher un nombre dans le texte
            number_match = re.search(r'(\d+)', text)
            if number_match:
                return int(number_match.group(1))
            return None
        except ValueError:
            return None
    
    def parse_markdown_table(self, table_section: str) -> List[Dict[str, Any]]:
        """
        Parse une section de tableau markdown générique.
        
        Args:
            table_section (str): Section contenant le tableau markdown
            
        Returns:
            List[Dict[str, Any]]: Liste des lignes parsées sous forme de dictionnaires
        """
        if not table_section:
            return []
        
        lines = table_section.strip().split('\n')
        headers = []
        data_rows = []
        
        # Trouver les en-têtes et les données
        header_found = False
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # Ligne d'en-tête
            if '|' in line and not header_found:
                headers = [h.strip() for h in line.split('|')[1:-1]]
                header_found = True
                continue
            
            # Ligne de séparation
            if header_found and line.startswith('|') and '-' in line:
                continue
            
            # Lignes de données
            if header_found and line.startswith('|'):
                columns = [col.strip() for col in line.split('|')[1:-1]]
                if len(columns) >= len(headers):
                    row_dict = {}
                    for i, header in enumerate(headers):
                        row_dict[header] = columns[i] if i < len(columns) else ""
                    data_rows.append(row_dict)
        
        return data_rows

## Tests du Parser

Testons le parser avec des exemples de données.

In [ ]:
# Test du parser avec un exemple
if __name__ == "__main__":
    sample_avis = """
## 1. LIVRES DISCUTÉS AU PROGRAMME du 15 décembre 2024

| Auteur | Titre | Éditeur | Avis détaillés des critiques | Note moyenne | Nb critiques | Coup de cœur | Chef d'œuvre |
|--------|-------|---------|------------------------------|--------------|--------------|-------------|-------------|
| Marie Darrieussecq | Kakapo | Gallimard | **Elisabeth Philippe**: "Magnifique" (9) <br>**Jérôme Garcin**: "Remarquable" (8) | <span style="background-color: #4CAF50; color: white; padding: 2px 6px; border-radius: 3px; font-weight: bold;">8.5</span> | 2 | Elisabeth Philippe | |
| Alice Zeniter | L'Art de perdre | Actes Sud | **Michel Crépu**: "Excellent" (9) <br>**Patricia Martin**: "Très bon" (7) | <span style="background-color: #4CAF50; color: white; padding: 2px 6px; border-radius: 3px; font-weight: bold;">8.0</span> | 2 | Michel Crépu | |

## 2. COUPS DE CŒUR DES CRITIQUES du 15 décembre 2024

| Auteur | Titre | Éditeur | Critique | Note | Commentaire |
|--------|-------|---------|----------|------|-------------|
| Kamel Daoud | Zabor ou les Psaumes | Actes Sud | Frédéric Beigbeder | <span style="background-color: #00C851; color: white; padding: 2px 6px; border-radius: 3px; font-weight: bold;">9.5</span> | Un livre bouleversant |
"""
    
    parser = AvisCritiquesParser()
    books = parser.extract_books_from_summary(sample_avis)
    
    print(f"Livres extraits: {len(books)}")
    for book in books:
        print(f"- {book.auteur}: {book.titre} ({book.section}) - Note: {book.note}")